# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratório 4

Neste laboratório exploraremos frameworks avançados para construção de aplicações inteligentes: **Semantic Kernel** e **AutoGen** com Azure OpenAI. Aprenderemos como criar agentes conversacionais, orchestrar múltiplos agentes, implementar plugins personalizados e desenvolver workflows complexos de automação.

O **Semantic Kernel** é um SDK open-source da Microsoft que permite integrar modelos de IA (como Azure OpenAI) com linguagens de programação convencionais, oferecendo funcionalidades como plugins, planejamento e memória.

O **AutoGen** é um framework da Microsoft para criar aplicações multi-agente onde diferentes agentes podem colaborar para resolver problemas complexos através de conversação estruturada.

### Pré-requisitos

Certifique-se de que as variáveis de ambiente estão configuradas no arquivo `.env` na raiz do repositório com suas credenciais do Azure OpenAI.

### Exercício 1 - Configuração Inicial e Semantic Kernel

Vamos começar instalando e importando as bibliotecas necessárias para trabalhar com Semantic Kernel e AutoGen.

In [ ]:
# Instalação das dependências necessárias
#%pip install semantic-kernel autogen-agentchat autogen-ext[openai,azure] python-dotenv

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.functions import KernelArguments
from semantic_kernel.prompt_template import PromptTemplateConfig

# Importações para AutoGen versão 0.6+
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Carregando variáveis de ambiente
load_dotenv(dotenv_path="../../../.env")

In [ ]:
# Configuração das credenciais do Azure OpenAI
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("Configurações carregadas:")
print(f"Endpoint: {azure_endpoint}")
print(f"Deployment: {deployment_name}")
print(f"API Version: {api_version}")

#### Configurando o Semantic Kernel

O Semantic Kernel atua como uma camada de orquestração que permite combinar modelos de IA com código convencional. Vamos criar um kernel e configurá-lo para usar o Azure OpenAI:

In [ ]:
# Inicializando o Semantic Kernel
kernel = sk.Kernel()

# Adicionando o serviço de chat do Azure OpenAI
chat_service = AzureChatCompletion(
    deployment_name=deployment_name,
    endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

kernel.add_service(chat_service)

print("Semantic Kernel configurado com sucesso!")

#### Primeiro Exemplo: Função Simples com Semantic Kernel

Vamos criar uma função simples que utiliza o Semantic Kernel para gerar conteúdo:

In [ ]:
# Criando uma função simples usando template de prompt
prompt_template = """
Você é um especialista em {{$topic}}. 
Crie um resumo informativo e conciso sobre {{$topic}} em {{$language}}.
O resumo deve ter aproximadamente {{$length}} palavras.

Tópico: {{$topic}}
"""

# Criando a função do kernel
summary_function = kernel.add_function(
    function_name="CreateSummary",
    plugin_name="ContentPlugin",
    prompt=prompt_template,
    description="Cria um resumo sobre qualquer tópico"
)

# Executando a função
async def run_summary_example():
    arguments = KernelArguments(
        topic="Inteligência Artificial",
        language="português", 
        length="150"
    )
    
    result = await kernel.invoke(summary_function, arguments)
    return result

# Executando o exemplo
result = await run_summary_example()
print("Resumo gerado pelo Semantic Kernel:")
print(result)

### Exercício 2 - Plugins e Funcionalidades Avançadas

#### Trabalhando com Plugins Built-in

O Semantic Kernel oferece plugins integrados que fornecem funcionalidades comuns. Vamos explorar o TextPlugin:

In [ ]:
# Adicionando plugin de texto built-in
kernel.add_plugin(TextPlugin(), plugin_name="text")

# Exemplo de uso do plugin de texto
text_to_process = """
O Azure AI Foundry é uma plataforma completa para desenvolvimento de aplicações de IA. 
Ela oferece ferramentas para treinar, implantar e gerenciar modelos de machine learning. 
Com integração nativa ao Azure OpenAI, desenvolvedores podem criar aplicações inteligentes rapidamente.
"""

async def demonstrate_text_plugin():
    # Função para fazer uppercase
    uppercase_result = await kernel.invoke(
        plugin_name="text",
        function_name="uppercase", 
        input=text_to_process
    )
    
    print("Texto em maiúsculas:")
    print(uppercase_result)
    print("\n" + "="*50 + "\n")
    
    

await demonstrate_text_plugin()

#### Criando um Plugin Personalizado

Vamos criar um plugin personalizado que simula operações matemáticas e de análise de dados:

In [ ]:
from semantic_kernel.functions import kernel_function
from typing import Annotated
import statistics

class MathPlugin:
    """Plugin personalizado para operações matemáticas"""
    
    @kernel_function(
        description="Calcula a média de uma lista de números",
        name="calculate_average"
    )
    def calculate_average(
        self, 
        numbers: Annotated[str, "Lista de números separados por vírgula"]
    ) -> Annotated[str, "A média dos números"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            average = statistics.mean(num_list)
            return f"A média dos números {numbers} é: {average:.2f}"
        except Exception as e:
            return f"Erro ao calcular média: {str(e)}"
    
    @kernel_function(
        description="Encontra o maior número em uma lista",
        name="find_maximum"
    )
    def find_maximum(
        self, 
        numbers: Annotated[str, "Lista de números separados por vírgula"]
    ) -> Annotated[str, "O maior número"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            maximum = max(num_list)
            return f"O maior número em {numbers} é: {maximum}"
        except Exception as e:
            return f"Erro ao encontrar máximo: {str(e)}"

# Adicionando o plugin personalizado ao kernel
math_plugin = MathPlugin()
kernel.add_plugin(math_plugin, plugin_name="math")

# Testando o plugin personalizado
async def test_custom_plugin():
    numbers = "10, 25, 15, 30, 8, 22"
    
    # Calculando média
    avg_result = await kernel.invoke(
        plugin_name="math",
        function_name="calculate_average",
        numbers=numbers
    )
    print(avg_result)
    
    # Encontrando máximo
    max_result = await kernel.invoke(
        plugin_name="math", 
        function_name="find_maximum",
        numbers=numbers
    )
    print(max_result)

await test_custom_plugin()

### Exercício 3 - Introdução ao AutoGen

O **AutoGen** é um framework poderoso para criar sistemas multi-agente onde diferentes agentes podem colaborar através de conversação estruturada. Diferente do Semantic Kernel que foca na orquestração de funções, o AutoGen permite criar agentes especializados que podem debater, colaborar e chegar a conclusões de forma autônoma.

#### Conceitos Fundamentais do AutoGen

- **Agentes**: Entidades que podem enviar e receber mensagens
- **Conversação**: Fluxo de mensagens entre agentes
- **Roles**: Papéis específicos dos agentes (Assistant, User Proxy, etc.)
- **Group Chat**: Conversas entre múltiplos agentes
- **Terminação**: Condições para finalizar conversas

#### Configuração Básica do AutoGen

**Nota importante**: Este exemplo usa a nova API do AutoGen (versão 0.6+) que substitui o `llm_config` pelo `model_client`. Certifique-se de usar `autogen-ext[openai,azure]` para ter acesso às extensões necessárias.

Vamos começar configurando os agentes básicos e criando uma conversação simples:

In [ ]:
# Configuração básica do AutoGen usando a nova API (versão 0.6+)
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Criando o cliente Azure OpenAI
azure_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=deployment_name,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

# Criando um agente assistente especializado usando a nova API
assistente_criativo = AssistantAgent(
    name="assistente_criativo",
    model_client=azure_model_client,
    system_message="""Você é um assistente criativo especializado em geração de ideias.
    Sua função é ajudar a brainstorming e criar conteúdo inovador.
    Seja criativo, mas mantenha suas respostas concisas e práticas."""
)

print("Agentes AutoGen configurados com sucesso!")
print(f"Assistente: {assistente_criativo.name}")
print("Modelo configurado para usar Azure OpenAI")

#### Exemplo Simples: Conversa entre Agentes

Vamos criar uma conversação simples entre o usuário e o assistente criativo para gerar ideias para um projeto:

In [ ]:
# Exemplo de conversa simples entre agentes usando a nova API
async def executar_exemplo_autogen():
    """
    Exemplo simples de como os agentes AutoGen podem colaborar
    para gerar ideias criativas para um projeto
    """
    
    # Mensagem inicial do usuário
    mensagem_inicial = """
    Preciso de ideias criativas para um aplicativo móvel que ajude pessoas 
    a organizar sua rotina diária. O aplicativo deve ser inovador e fácil de usar.
    
    Por favor, me dê 3 ideias principais com uma breve descrição de cada uma.
    """
    
    # Executando o agente
    try:
        print("=== Iniciando Conversa com AutoGen ===\n")
        
        # Usando a nova API - método run
        resultado = await assistente_criativo.run(task=mensagem_inicial)
        
        print("Resposta do assistente:")
        # A nova API retorna um TaskResult com mensagens
        if resultado.messages:
            for message in resultado.messages:
                if hasattr(message, 'content'):
                    print(f"- {message.content}")
        
        print("\n=== Conversa Finalizada ===")
        return resultado
        
    except Exception as e:
        print(f"Erro durante a conversa: {str(e)}")
        return None

# Executando o exemplo
resultado_conversa = await executar_exemplo_autogen()

#### Entendendo o Que Aconteceu

No exemplo acima, demonstramos os conceitos fundamentais do AutoGen na versão 0.6+:

1. **Configuração dos Agentes**: Criamos um agente utilizando a nova API:
   - `AssistantAgent`: Especializado em criatividade e geração de ideias
   - Usa `model_client` em vez do antigo `llm_config`

2. **Método `run()`**: A nova API usa o método `run()` que:
   - Aceita uma task como string
   - Retorna um `TaskResult` com as mensagens
   - É assíncrono por padrão

3. **Especialização**: O agente tem uma `system_message` que define seu papel e comportamento específico.

4. **Simplicidade**: A nova API é mais limpa e focada em casos de uso específicos.

### Melhores Práticas e Casos de Uso Avançados

#### Quando Usar Semantic Kernel vs AutoGen

**Semantic Kernel é ideal para:**
- Orchestração de AI com código tradicional
- Criação de plugins e funções reutilizáveis
- Aplicações que precisam de planejamento automático
- Integração com sistemas existentes
- Aplicações single-agent com funcionalidades complexas

**AutoGen é ideal para:**
- Sistemas multi-agente complexos
- Simulações de discussões e brainstorming
- Workflows que envolvem múltiplas perspectivas
- Automação de processos colaborativos
- Sistemas de tomada de decisão distribuída

#### Casos de Uso Práticos

1. **Sistema de Atendimento ao Cliente**
   - Semantic Kernel: Criação de plugins para consulta de pedidos, produtos, etc.
   - AutoGen: Agentes especializados (triagem, vendas, suporte técnico)

2. **Análise de Documentos Corporativos**
   - Semantic Kernel: Funções para extração e processamento de texto
   - AutoGen: Agentes revisores, analistas e especialistas em domínio

3. **Planejamento de Projetos**
   - Semantic Kernel: Plugins para cálculos de cronograma e recursos
   - AutoGen: Agentes representando diferentes stakeholders

4. **Sistema de Recomendações**
   - Semantic Kernel: Funções de filtragem e cálculo de similaridade
   - AutoGen: Agentes com diferentes critérios de recomendação

In [ ]:
# Exemplo Final: Sistema de Análise de Produtos
# Combinando SK e AutoGen (versão 0.6+) para análise de feedback de produtos

# Plugin do SK para análise de sentimentos (simulado)
class SentimentPlugin:
    @kernel_function(
        description="Analisa o sentimento de um texto",
        name="analyze_sentiment"
    )
    def analyze_sentiment(
        self, 
        text: Annotated[str, "Texto para análise de sentimento"]
    ) -> Annotated[str, "Resultado da análise de sentimento"]:
        # Simulação simples de análise de sentimento
        palavras_positivas = ["excelente", "ótimo", "bom", "recomendo", "perfeito"]
        palavras_negativas = ["ruim", "péssimo", "horrível", "não recomendo", "defeito"]
        
        text_lower = text.lower()
        score_positivo = sum(1 for palavra in palavras_positivas if palavra in text_lower)
        score_negativo = sum(1 for palavra in palavras_negativas if palavra in text_lower)
        
        if score_positivo > score_negativo:
            return f"POSITIVO (Score: +{score_positivo - score_negativo})"
        elif score_negativo > score_positivo:
            return f"NEGATIVO (Score: -{score_negativo - score_positivo})"
        else:
            return "NEUTRO (Score: 0)"

# Adicionando o plugin ao kernel
sentiment_plugin = SentimentPlugin()
kernel.add_plugin(sentiment_plugin, plugin_name="sentiment")

# Criando agentes especializados para análise de produtos usando a nova API
agente_qualidade = AssistantAgent(
    name="especialista_qualidade",
    model_client=azure_model_client,
    system_message="""Você é um especialista em qualidade de produtos.
    Analise feedbacks focando em aspectos de qualidade, durabilidade e design.
    Identifique padrões e sugira melhorias."""
)

agente_experiencia = AssistantAgent(
    name="especialista_ux",
    model_client=azure_model_client,
    system_message="""Você é um especialista em experiência do usuário.
    Analise feedbacks focando na usabilidade, facilidade de uso e satisfação.
    Sugira melhorias na experiência do cliente."""
)

# Exemplo de execução do sistema completo
async def sistema_analise_produtos():
    feedbacks = [
        "O produto é excelente! Muito fácil de usar e a qualidade surpreendeu.",
        "Tive problemas com o produto. Não funciona como esperado, péssima experiência.",
        "O design é bom mas poderia melhorar a usabilidade. Recomendo com ressalvas."
    ]
    
    print("=== Sistema de Análise de Produtos ===\n")
    
    # Análise de sentimentos usando Semantic Kernel
    analises_sentimento = []
    for i, feedback in enumerate(feedbacks, 1):
        sentimento = await kernel.invoke(
            plugin_name="sentiment",
            function_name="analyze_sentiment",
            text=feedback
        )
        analises_sentimento.append(f"Feedback {i}: {sentimento}")
        print(f"Feedback {i}: {feedback}")
        print(f"Sentimento: {sentimento}\n")
    
    # Preparando relatório consolidado
    relatorio_consolidado = f"""
    Análise de Feedbacks de Produtos - Relatório Consolidado
    
    Feedbacks analisados:
    {chr(10).join(f'{i+1}. {feedback}' for i, feedback in enumerate(feedbacks))}
    
    Análises de sentimento:
    {chr(10).join(analises_sentimento)}
    
    Por favor, forneça sua análise especializada e recomendações para melhorar o produto.
    """
    
    # Análise do especialista em qualidade
    print("=== Análise do Especialista em Qualidade ===")
    resultado_qualidade = await agente_qualidade.run(task=relatorio_consolidado)
    if resultado_qualidade.messages:
        print(resultado_qualidade.messages[-1].content)
    
    print("\n=== Análise do Especialista em UX ===")
    resultado_ux = await agente_experiencia.run(task=relatorio_consolidado)
    if resultado_ux.messages:
        print(resultado_ux.messages[-1].content)

# Executar o sistema (descomente para testar)
# await sistema_analise_produtos()

print("Sistema de análise de produtos configurado!")
print("Descomente a linha 'await sistema_analise_produtos()' para executar o exemplo completo.")

### Conclusão e Próximos Passos

Neste laboratório, exploramos as poderosas capacidades do **Semantic Kernel** e **AutoGen** (versão 0.6+) para criar aplicações inteligentes avançadas com Azure OpenAI. Aprendemos:

#### Semantic Kernel
- ✅ Configuração e inicialização do kernel
- ✅ Criação e uso de funções personalizadas
- ✅ Trabalho com plugins built-in e personalizados
- ✅ Templates de prompts e argumentos dinâmicos

#### AutoGen
- ✅ Configuração de agentes com a API
- ✅ Uso do `model_client` em vez de `llm_config`
- ✅ Método `run()` assíncrono para execução
- ✅ Especialização de agentes por domínio


### Recursos Adicionais

Para aprofundar seus conhecimentos, explore os seguintes recursos:

#### Semantic Kernel
- [Documentação Oficial do Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Repositório GitHub do Semantic Kernel](https://github.com/microsoft/semantic-kernel)
- [Exemplos e Tutoriais](https://learn.microsoft.com/en-us/semantic-kernel/get-started/)
- [Plugins da Comunidade](https://github.com/microsoft/semantic-kernel/tree/main/python/semantic_kernel/core_plugins)

#### AutoGen
- [Documentação Oficial do AutoGen](https://microsoft.github.io/autogen/)
- [Repositório GitHub do AutoGen](https://github.com/microsoft/autogen)
- [Galeria de Exemplos](https://microsoft.github.io/autogen/docs/Examples)
- [Notebooks de Tutorial](https://github.com/microsoft/autogen/tree/main/notebook)

#### Azure AI Foundry
- [Portal do Azure AI Foundry](https://ai.azure.com/)
- [Documentação do Azure OpenAI](https://learn.microsoft.com/en-us/azure/ai-services/openai/)
- [Modelos Disponíveis](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/foundry-models-overview)
- [Melhores Práticas](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/best-practices)

### Desafios para Prática

1. **Expansão do Sistema de Produtos**: Adicione mais plugins ao Semantic Kernel para análise de imagens de produtos
2. **Agentes Especializados**: Crie agentes para diferentes setores (finanças, administrativo, educação)
3. **Workflow Complexo**: Implemente um sistema de aprovação multi-etapas usando AutoGen
4. **Interface de Usuário**: Crie uma interface web para interagir com seus agentes